In [12]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf

In [3]:
mid = MidiFile('LSTM/Samples/DPR LIVE-9-NEON.mid') 
notes = []

notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])
        
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 30
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

In [7]:
len(X_test)

300

In [17]:
def Generator():
    inputs = tf.keras.layers.Input(shape=[None, 256])
    x = LSTM(300, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = LSTM(300, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = LSTM(300, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = LSTM(300, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = Dense(300)(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = x)
    
    optimizer = Adam(lr=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    
    return model
    

In [15]:
def Discriminator():
    inputs = tf.keras.layers.Input(shape=[None, 300])
    x = LSTM(256, input_shape=(300, 1), return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = LSTM(128, input_shape=(300, 1), return_sequences=True)(x)
    x = Dropout(0.6)(x)
    x = LSTM(64, input_shape=(300, 1), return_sequences=False)(x)
    x = Dropout(0.6)(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = x)
    optimizer = Adam(lr=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

In [18]:
generator = Generator()

In [19]:
discriminator = Discriminator()

In [20]:
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, 300)]       0         
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 256)         570368    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 128)         197120    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0   

In [21]:
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, 256)]       0         
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 300)         668400    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
dense (Dense)                (None, None, 300)         90300     
Total params: 758,700
Trainable params: 758,700
Non-trainable params: 0
_________________________________________________________________


In [22]:
z_vector = tf.keras.models.load_model('LSTM/Checkpoints/checkpoint_model_05.hdf5')

In [24]:
z_vector.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 256)           264192    
_________________________________________________________________
dropout (Dropout)            (None, 30, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6

In [25]:
print(z_vector.layers[0].weights)

[<tf.Variable 'lstm/lstm_cell_9/kernel:0' shape=(1, 1024) dtype=float32, numpy=
array([[-1.0772903 , -1.7238122 , -1.2400359 , ..., -1.9222625 ,
         0.25038102, -1.6269035 ]], dtype=float32)>, <tf.Variable 'lstm/lstm_cell_9/recurrent_kernel:0' shape=(256, 1024) dtype=float32, numpy=
array([[-0.9175383 , -0.4392479 , -0.42765284, ..., -0.38742852,
        -0.3573955 , -0.95983607],
       [-0.37212878,  0.23964775,  0.11669407, ...,  0.18710889,
        -0.3872067 , -0.30723503],
       [ 0.3669471 ,  0.10972648,  0.14692955, ...,  0.04475506,
         0.6147749 ,  0.2743186 ],
       ...,
       [ 0.42170334,  0.04790486,  0.19503649, ...,  0.13895783,
         0.55621827,  0.40762404],
       [ 0.8026749 ,  0.252006  ,  0.84107524, ..., -0.04176012,
         0.18649848,  0.8964987 ],
       [-0.1712545 , -0.08709455,  0.03232609, ..., -0.03480397,
        -0.19071646, -0.16793591]], dtype=float32)>, <tf.Variable 'lstm/lstm_cell_9/bias:0' shape=(1024,) dtype=float32, numpy=
array(

In [32]:
z_vector_weights = z_vector.layers[0].weights